Cross check the results from Higgs combine and the results from the dftools.fitting implementation

In [1]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import numdifftools
import iminuit
import dftools

Welcome to JupyROOT 6.18/00


In [2]:
df = pd.read_csv("/vols/build/cms/sdb15/ZinvWidth/HiggsCombine/CMSSW_10_2_13/src/HiggsAnalysis/CombinedLimit/data/tutorials/shapes/simple-shapes-df_input.csv")
df.columns = ["region", "process", "variation", "bin_min", "sum_w", "sum_ww"]
df["bin_max"] = df["bin_min"]+1.
df = df.set_index(["region", "process", "variation", "bin_min", "bin_max"])
df_data = df.loc[(
    (df.index.get_level_values("region")=="bin1")
    & (df.index.get_level_values("process")=="data_obs")
)].groupby(["region", "bin_min", "bin_max"]).sum()

df_mc = df.loc[(
    (df.index.get_level_values("region")=="bin1")
    & (df.index.get_level_values("process").isin(["signal", "background"]))
)].reset_index()
df_mc.loc[df_mc["variation"]=="nominal", "variation"] = ""
df_mc = df_mc.groupby(["region", "process", "variation", "bin_min", "bin_max"]).sum()

tdf_mc = df_mc.loc[
    (df_mc.index.get_level_values("process")=="signal")
    & (df_mc.index.get_level_values("variation")==""), :
].reset_index()
tdf_mc.loc[:,"variation"] = "lumiUp"
df_mc = pd.concat([df_mc, tdf_mc.set_index(["region", "process", "variation", "bin_min", "bin_max"])*1.1], sort=True)
tdf_mc.loc[:,"variation"] = "lumiDown"
df_mc = pd.concat([df_mc, tdf_mc.set_index(["region", "process", "variation", "bin_min", "bin_max"])/1.1], sort=True)

tdf_mc = df_mc.loc[
    (df_mc.index.get_level_values("process")=="background")
    & (df_mc.index.get_level_values("variation")==""), :
].reset_index()
tdf_mc.loc[:,"variation"] = "bgnormUp"
df_mc = pd.concat([df_mc, tdf_mc.set_index(["region", "process", "variation", "bin_min", "bin_max"])*1.3], sort=True)
tdf_mc.loc[:,"variation"] = "bgnormDown"
df_mc = pd.concat([df_mc, tdf_mc.set_index(["region", "process", "variation", "bin_min", "bin_max"])/1.3], sort=True)

df_data = df_data.iloc[:-1]
df_mc = pd.pivot_table(df_mc, index=["bin_min", "bin_max"], values=["sum_w", "sum_ww"], columns=["region", "process", "variation"]).iloc[:-1]
df_mc = df_mc.stack().stack().stack().reset_index().set_index(["region", "process", "variation", "bin_min", "bin_max"]).sort_index()

bins = (df_data.index.get_level_values("bin_min").values, df_data.index.get_level_values("bin_max").values)

config = {
    "regions": {"bin1": ["signal", "background"]},
    "parameters": [
        {"name": "rsignal",          "value": 0., "limit": (-5., 5.),    "fixed": False, "constraint": "free"},
        {"name": "rbackground",      "value": 1., "limit": (None, None), "fixed": True,  "constraint": "free"},
        {"name": "alpha",            "value": 0., "limit": (-3., 3.),    "fixed": False, "constraint": "gaussian"},
        {"name": "sigma",            "value": 0., "limit": (-3., 3.),    "fixed": False, "constraint": "gaussian"},
        {"name": "lumi",             "value": 0., "limit": (-3., 3.),    "fixed": False, "constraint": "gaussian"},
        {"name": "bgnorm",           "value": 0., "limit": (-3., 3.),    "fixed": False, "constraint": "gaussian"},
        {"name": "bin1_mcstat_bin0", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin1", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin2", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin3", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin4", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin5", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin6", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin7", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin8", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
        {"name": "bin1_mcstat_bin9", "value": 0., "limit": (-10., 10.),  "fixed": False, "constraint": "gamma"},
    ],
    "scale_functions": {
        ("bin1", "signal"):     "x, w, p: p['rsignal']",
        ("bin1", "background"): "x, w, p: p['rbackground']",
    }
}

In [4]:
model = dftools.fitting.NLLModel(
    df_data, df_mc, config, shape_type="shapeN",
)
minimizer = model.fit(migrad=False, minos=False)
minimizer.migrad()
minimizer.minos()
minimizer.migrad()

------------------------------------------------------------------
| FCN = 86.16                   |     Ncalls=92 (6504 total)     |
| EDM = 2.21E-10 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
|   | Name             |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------------------
| 0 | rsignal          |    0.17   |    0.66   |   -0.68    |    0.76    |   -5    |    5    |       |
| 1 | rbackground      |    1.0    |    1.0    |            |            |         |         |  yes  |
| 2 | alpha            |   -0.18   |    0.68   |   -0.69    |    0.66    |   -3    |    3    |       |
| 3 | sigma            |  -0.015   |   0.987   |   -0.997   |    0.996   |   -3    |    3    |       |
| 4 | lumi             |   0.000   |   0.982   |   -1.000   |    1.000   |   -3    |    3    |       |
| 5 | bgnorm           |   -0.5    |    0.5    |    -0.5    |     0.5    |   -3    |    3    |       |
| 6 | bin1_mcstat_bin0 |   -0.29   |    0.93   |   -0.95    |    0.92    |   -10   |   10    |       |
| 7 | bin1_mcstat_bin1 |   -0.5    |    0.9    |    -0.9    |     0.9    |   -10   |   10    |       |
| 8 | bin1_mcstat_bin2 |    0.4    |    0.8    |    -0.9    |     0.8    |   -10   |   10    |       |
| 9 | bin1_mcstat_bin3 |    0.6    |    0.8    |    -0.9    |     0.8    |   -10   |   10    |       |
| 10| bin1_mcstat_bin4 |   -0.5    |    1.0    |    -1.0    |     0.9    |   -10   |   10    |       |
| 11| bin1_mcstat_bin5 |   -0.15   |    1.02   |   -1.07    |    0.99    |   -10   |   10    |       |
| 12| bin1_mcstat_bin6 |    0.19   |    1.03   |   -1.08    |    1.00    |   -10   |   10    |       |
| 13| bin1_mcstat_bin7 |    0.04   |    0.99   |   -1.05    |    0.94    |   -10   |   10    |       |
| 14| bin1_mcstat_bin8 |   0.015   |   1.038   |   -1.114   |    0.972   |   -10   |   10    |       |
| 15| bin1_mcstat_bin9 |   -1.0    |    1.4    |    -1.6    |     1.3    |   -10   |   10    |       |
------------------------------------------------------------------------------------------------------

In [11]:
model = dftools.fitting.NLLModel(
    df_data, df_mc, config, #shape_type="shapeN",
)
minimizer = model.fit(migrad=False, minos=False)
minimizer.migrad()
minimizer.minos()
minimizer.migrad()

------------------------------------------------------------------
| FCN = 86.16                   |     Ncalls=97 (6607 total)     |
| EDM = 6.23E-11 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
|   | Name             |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------------------
| 0 | rsignal          |    0.16   |    0.64   |   -0.65    |    0.73    |   -5    |    5    |       |
| 1 | rbackground      |    1.0    |    1.0    |            |            |         |         |  yes  |
| 2 | alpha            |   -0.16   |    0.63   |   -0.69    |    0.61    |   -3    |    3    |       |
| 3 | sigma            |  -0.016   |   0.982   |   -0.998   |    0.996   |   -3    |    3    |       |
| 4 | lumi             |   0.000   |   0.982   |   -1.000   |    1.000   |   -3    |    3    |       |
| 5 | bgnorm           |   -0.5    |    0.5    |    -0.5    |     0.5    |   -3    |    3    |       |
| 6 | bin1_mcstat_bin0 |   -0.29   |    0.93   |   -0.95    |    0.91    |   -10   |   10    |       |
| 7 | bin1_mcstat_bin1 |   -0.5    |    0.9    |    -0.9    |     0.9    |   -10   |   10    |       |
| 8 | bin1_mcstat_bin2 |    0.4    |    0.8    |    -0.9    |     0.8    |   -10   |   10    |       |
| 9 | bin1_mcstat_bin3 |    0.6    |    0.8    |    -0.9    |     0.8    |   -10   |   10    |       |
| 10| bin1_mcstat_bin4 |   -0.5    |    1.0    |    -1.0    |     0.9    |   -10   |   10    |       |
| 11| bin1_mcstat_bin5 |   -0.15   |    1.02   |   -1.07    |    0.98    |   -10   |   10    |       |
| 12| bin1_mcstat_bin6 |    0.20   |    1.03   |   -1.08    |    0.99    |   -10   |   10    |       |
| 13| bin1_mcstat_bin7 |    0.04   |    0.99   |   -1.05    |    0.94    |   -10   |   10    |       |
| 14| bin1_mcstat_bin8 |   0.007   |   1.026   |   -1.105   |    0.959   |   -10   |   10    |       |
| 15| bin1_mcstat_bin9 |   -1.0    |    1.4    |    -1.6    |     1.2    |   -10   |   10    |       |
------------------------------------------------------------------------------------------------------

In [ ]:
model = dftools.fitting.NLLModel(
    df_data, df_mc, bins, config, shape_type="shapeN"
)
minimizer = model.fit(migrad=False, minos=False)
minimizer.migrad()

In [23]:
minimizer.migrad()

------------------------------------------------------------------
| FCN = 86.16                   |     Ncalls=97 (6606 total)     |
| EDM = 7.13E-12 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------------------
|   | Name             |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------------------
| 0 | rsignal          |    0.17   |    0.66   |   -0.67    |    0.76    |   -5    |    5    |       |
| 1 | rbackground      |    1.0    |    1.0    |            |            |         |         |  yes  |
| 2 | alpha            |   -0.18   |    0.68   |   -0.69    |    0.66    |   -3    |    3    |       |
| 3 | sigma            |  -0.015   |   0.986   |   -1.000   |    0.994   |   -3    |    3    |       |
| 4 | lumi             |  -0.000   |   0.982   |   -1.000   |    1.000   |   -3    |    3    |       |
| 5 | bgnorm           |   -0.5    |    0.5    |    -0.5    |     0.5    |   -3    |    3    |       |
| 6 | bin1_mcstat_bin0 |   -0.29   |    0.93   |   -0.95    |    0.92    |   -10   |   10    |       |
| 7 | bin1_mcstat_bin1 |   -0.5    |    0.9    |    -0.9    |     0.9    |   -10   |   10    |       |
| 8 | bin1_mcstat_bin2 |    0.4    |    0.8    |    -0.9    |     0.8    |   -10   |   10    |       |
| 9 | bin1_mcstat_bin3 |    0.6    |    0.8    |    -0.9    |     0.8    |   -10   |   10    |       |
| 10| bin1_mcstat_bin4 |   -0.5    |    1.0    |    -1.0    |     0.9    |   -10   |   10    |       |
| 11| bin1_mcstat_bin5 |   -0.15   |    1.02   |   -1.07    |    0.98    |   -10   |   10    |       |
| 12| bin1_mcstat_bin6 |    0.19   |    1.03   |   -1.08    |    0.99    |   -10   |   10    |       |
| 13| bin1_mcstat_bin7 |    0.04   |    0.99   |   -1.05    |    0.94    |   -10   |   10    |       |
| 14| bin1_mcstat_bin8 |   0.015   |   1.038   |   -1.114   |    0.972   |   -10   |   10    |       |
| 15| bin1_mcstat_bin9 |   -1.0    |    1.4    |    -1.6    |     1.3    |   -10   |   10    |       |
------------------------------------------------------------------------------------------------------

In [22]:
minimizer.minos()

-------------------------------------------------
|     rsignal     |            Valid            |
-------------------------------------------------
|      Error      |     -0.7     |      0.8     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False     |
-------------------------------------------------
-------------------------------------------------
|      alpha      |            Valid            |
-------------------------------------------------
|      Error      |     -0.7     |      0.7     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False     |
-------------------------------------------------
-------------------------------------------------
|      sigma      |            Valid            |
-------------------------------------------------
|      Error      |     -1.0     |      1.0     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False     |
-------------------------------------------------
-------------------------------------------------
|      lumi       |            Valid            |
-------------------------------------------------
|      Error      |     -1.0     |      1.0     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False     |
-------------------------------------------------
-------------------------------------------------
|     bgnorm      |            Valid            |
-------------------------------------------------
|      Error      |     -0.5     |      0.5     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False     |
-------------------------------------------------
-------------------------------------------------
| bin1_mcstat_bin0 |            Valid            |
-------------------------------------------------
|      Error      |     -1.0     |      0.9     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False     |
-------------------------------------------------
-------------------------------------------------
| bin1_mcstat_bin1 |            Valid            |
-------------------------------------------------
|      Error      |     -0.9     |      0.9     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False     |
-------------------------------------------------
-------------------------------------------------
| bin1_mcstat_bin2 |            Valid            |
-------------------------------------------------
|      Error      |     -0.9     |      0.8     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False     |
-------------------------------------------------
-------------------------------------------------
| bin1_mcstat_bin3 |            Valid            |
-------------------------------------------------
|      Error      |     -0.9     |      0.8     |
|      Valid      |     True     |     True     |
|    At Limit     |    False     |    False     |
|     Max FCN     |    False     |    False     |
|     New Min     |    False     |    False   